# DashVector

> [DashVector](https://help.aliyun.com/document_detail/2510225.html) 是一个全托管的向量数据库服务，支持高维稠密向量和稀疏向量，以及实时插入和过滤搜索。它旨在自动扩展，并能适应不同的应用需求。

本 Notebook 展示了如何使用与 `DashVector` 向量数据库相关的各项功能。

要使用 DashVector，必须拥有一个 API 密钥。
以下是 [安装说明](https://help.aliyun.com/document_detail/2510223.html)。

## 安装

In [ ]:
%pip install --upgrade --quiet  langchain-community dashvector dashscope

我们要使用 `DashScopeEmbeddings`，因此我们也需要获取 Dashscope API 密钥。

In [1]:
import getpass
import os

if "DASHVECTOR_API_KEY" not in os.environ:
    os.environ["DASHVECTOR_API_KEY"] = getpass.getpass("DashVector API Key:")
if "DASHSCOPE_API_KEY" not in os.environ:
    os.environ["DASHSCOPE_API_KEY"] = getpass.getpass("DashScope API Key:")

## 示例

In [1]:
from langchain_community.embeddings.dashscope import DashScopeEmbeddings
from langchain_community.vectorstores import DashVector
from langchain_text_splitters import CharacterTextSplitter

In [2]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("../../how_to/state_of_the_union.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embeddings = DashScopeEmbeddings()

我们可以从文档创建新的 DashVector。

In [ ]:
dashvector = DashVector.from_documents(docs, embeddings)

query = "What did the president say about Ketanji Brown Jackson"
docs = dashvector.similarity_search(query)
print(docs)

我们可以添加带有元数据和 ID 的文本，并通过元数据过滤器进行搜索。

In [4]:
texts = ["foo", "bar", "baz"]
metadatas = [{"key": i} for i in range(len(texts))]
ids = ["0", "1", "2"]

dashvector.add_texts(texts, metadatas=metadatas, ids=ids)

docs = dashvector.similarity_search("foo", filter="key = 2")
print(docs)

[Document(page_content='baz', metadata={'key': 2})]


### 操作频段 `partition` 参数

`partition` 参数的默认值为 `default`，如果传入不存在的 `partition` 参数，则会自动创建该 `partition`。

In [ ]:
texts = ["foo", "bar", "baz"]
metadatas = [{"key": i} for i in range(len(texts))]
ids = ["0", "1", "2"]
partition = "langchain"

# add texts
dashvector.add_texts(texts, metadatas=metadatas, ids=ids, partition=partition)

# similarity search
query = "What did the president say about Ketanji Brown Jackson"
docs = dashvector.similarity_search(query, partition=partition)

# delete
dashvector.delete(ids=ids, partition=partition)